**Final Model**

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

c:\Users\alsol\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\alsol\miniconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
c:\Users\alsol\miniconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:

data = pd.read_csv('dataset/train_test_4class_no_empty_fixed_helpful_lower_punch_stopwords_emotion_outliers_eng.csv', encoding='latin-1', na_filter=True)


columns_to_drop = ['OverallScore']
data.drop(columns=columns_to_drop, inplace=True)


data.dropna(subset=['label'], inplace=True)


data = data.sample(frac=1, random_state=42).reset_index(drop=True)


train_data, rest_data = train_test_split(data, test_size=0.2, random_state=42, stratify=data['label'])


validation_data, test_data = train_test_split(rest_data, test_size=0.5, random_state=42, stratify=rest_data['label'])

In [3]:
class_support_train = train_data['label'].value_counts()
print(class_support_train)

label
3    3197
2     667
4     518
1     198
Name: count, dtype: int64


In [4]:
class_support_validation = validation_data['label'].value_counts()
print(class_support_validation)

label
3    399
2     83
4     65
1     25
Name: count, dtype: int64


In [5]:
class_support_test = test_data['label'].value_counts()
print(class_support_test)

label
3    400
2     84
4     65
1     24
Name: count, dtype: int64


In [4]:

train_data.reset_index(drop=True, inplace=True)
validation_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [6]:
print(len(train_data))

5364


In [7]:

label = LabelEncoder()
y_train = label.fit_transform(train_data['label'])
y_train = to_categorical(y_train)
y_validation = label.transform(validation_data['label'])
y_validation = to_categorical(y_validation)
y_test = label.transform(test_data['label'])
y_test = to_categorical(y_test)

In [12]:
print(len(y_train))

5210


In [13]:
from transformers import AutoTokenizer, TFConvBertForSequenceClassification

In [14]:
# tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

tokenizer = AutoTokenizer.from_pretrained("YituTech/conv-bert-base")


In [8]:
# Changed the tokenizer to ALBERT
# tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

convBert_model_layer = TFConvBertForSequenceClassification.from_pretrained("YituTech/conv-bert-base")

All model checkpoint layers were used when initializing TFConvBertForSequenceClassification.

Some layers of TFConvBertForSequenceClassification were not initialized from the model checkpoint at YituTech/conv-bert-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
# Initial One

def convBert_encode(data, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for i in range(len(data)):
        text = (str(data['ReviewerName'][i]) + " " +
                str(data['ProductTitle'][i]) + " " +
                str(data['ProductDetails'][i]) + " " +
                str(data['ReviewTitle'][i]) + " " +
                str(data['ReviewDetails'][i]))
        
        numerical_columns = [str(data[column][i]) for column in ['ReviewRating', 
                                                                 'VerifiedPurchase', 'HelpfulVote',
                                                                 'ImageStatus', 'VineReviewStatus']]
        
        text = tokenizer.tokenize(text) + numerical_columns
        
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)



In [11]:
def convBert_encode(data, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for i in range(len(data)):
        text = (str(data['ReviewerName'][i]) + " " +
                str(data['ProductTitle'][i]) + " " +
                str(data['ProductDetails'][i]) + " " +
                str(data['ReviewTitle'][i]) + " " +
                str(data['ReviewDetails'][i]))
        
        numerical_columns = [str(data[column][i]) for column in ['ReviewRating', 
                                                                 'VerifiedPurchase', 'HelpfulVote',
                                                                 'ImageStatus', 'VineReviewStatus']]
        
        text = text + " " + " ".join(numerical_columns)
        print(text)
        
        inputs = tokenizer.encode_plus(text, add_special_tokens=True, max_length=max_len, 
                                       padding='max_length', truncation=True)
        
        all_tokens.append(inputs['input_ids'])
        all_masks.append(inputs['attention_mask'])
        all_segments.append(inputs['token_type_ids'])
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
max_len = 250
albert_train_input = convBert_encode(train_data, tokenizer, max_len=max_len)
albert_validation_input = convBert_encode(validation_data, tokenizer, max_len=max_len)
albert_test_input = convBert_encode(test_data, tokenizer, max_len=max_len)

In [13]:
# Stacked bidirectional LSTM with both sequence and pooled outputs
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l1_l2

def build_convBert_model(convBert_model,  max_len=250):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    

    convBert_outputs = convBert_model(input_word_ids, attention_mask=input_mask, token_type_ids=segment_ids)
    convBert_sequence_output = convBert_outputs[0]
    convBert_sequence_output = tf.tile(tf.expand_dims(convBert_sequence_output, axis=1), [1, 512, 1])
    

    lstm_output = Bidirectional(LSTM(256, return_sequences=True))(convBert_sequence_output)
    lstm_output = Bidirectional(LSTM(256, return_sequences=True))(lstm_output)
    lstm_output = Bidirectional(LSTM(256))(lstm_output)
    
    lay = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(lstm_output)
    lay = Dropout(0.2)(lay) # make dropout layer 0.5 to decrease the train accuracy
    lay = Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(lay)
    lay = Dropout(0.2)(lay)
    lay = Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(lay)
    lay = Dropout(0.2)(lay)
    out = Dense(3, activation='softmax', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(lay)

    # lay = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(lstm_output)
    # lay = tf.keras.layers.Dropout(0.2)(lay)
    # lay = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(lay)
    # lay = tf.keras.layers.Dropout(0.2)(lay)
    # lay = tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(lay)
    # lay = tf.keras.layers.Dropout(0.2)(lay)
    # out = tf.keras.layers.Dense(4, activation='softmax', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(lay)
    
    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [27]:
print(len(albert_train_input[0]) == len(y_train))

True


In [15]:
convBert_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 512)]        0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 512)]        0           []                               
                                                                                                  
 tf_conv_bert_for_sequence_clas  TFSequenceClassifie  106272650  ['input_word_ids[0][0]',         
 sification (TFConvBertForSeque  rOutput(loss=None,               'input_mask[0][0]',         

Trying numerical

In [15]:
def convBert_encode_text_only(data, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    all_numerical_data = []
                    # str(data['ProfileID'][i]) + " " +
    for i in range(len(data)): # keeping ReviewDetails at first so even if it gets Truncated, ReviewDetails will still the there at most
        text = (str(data['ReviewDetails'][i]) + " " +
                str(data['ProductDetails'][i]) + " " +
                str(data['ProductTitle'][i]) + " " +
                str(data['ReviewTitle'][i]) + " " +
                str(data['Date'][i]) + " " +
                str(data['ReviewerName'][i]))

        
        numerical_data = [data[column][i] for column in ['ReviewRating', 'HelpfulVote', 'ProductPrice',
                                                         'VerifiedPurchase',
                                                         'ImageStatus', 'VineReviewStatus']]
        
        inputs = tokenizer.encode_plus(text, add_special_tokens=True, max_length=max_len, 
                                       padding='max_length', truncation=True)
        
        all_tokens.append(inputs['input_ids'])
        all_masks.append(inputs['attention_mask'])
        all_segments.append(inputs['token_type_ids'])
        all_numerical_data.append(numerical_data)
        
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments), np.array(all_numerical_data)

In [16]:
train_data.reset_index(drop=True, inplace=True)
validation_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [17]:
max_len = 512
# albert_train_input = convBert_encode_text_only(train_data, tokenizer, max_len=max_len)
albert_train_input = convBert_encode_text_only(train_data, tokenizer, max_len=max_len)
albert_validation_input = convBert_encode_text_only(validation_data, tokenizer, max_len=max_len)
albert_test_input = convBert_encode_text_only(test_data, tokenizer, max_len=max_len)

In [21]:
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Bidirectional, LSTM

# robust model with many columns and numerical data

def build_convBert_model_kfolds(convBert_model, num_numerical_features, max_len=250):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    numerical_input = Input(shape=(num_numerical_features,), name="numerical_input")
    

    convBert_outputs = convBert_model(input_word_ids, attention_mask=input_mask, token_type_ids=segment_ids)
    convBert_sequence_output = convBert_outputs[0]
    convBert_sequence_output = tf.tile(tf.expand_dims(convBert_sequence_output, axis=1), [1, 512, 1])
    

    lstm_output = Bidirectional(LSTM(256, return_sequences=True))(convBert_sequence_output)
    lstm_output = Bidirectional(LSTM(256, return_sequences=True))(lstm_output)
    lstm_output = Bidirectional(LSTM(256))(lstm_output)
    

    numerical_output = Dense(32, activation='relu')(numerical_input)
    numerical_output = Dropout(0.2)(numerical_output)
    

    concatenated = concatenate([lstm_output, numerical_output])
    
    lay = Dense(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(concatenated)
    lay = Dropout(0.2)(lay)
    lay = Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(lay)
    lay = Dropout(0.2)(lay)
    lay = Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(lay)
    lay = Dropout(0.2)(lay)
    out = Dense(3, activation='softmax', kernel_regularizer=l1_l2(l1=0.01, l2=0.01))(lay)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids, numerical_input], outputs=out)
    model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

Main convBert Code

In [18]:
from transformers import TFAutoModel
convBert_model_layer = TFAutoModel.from_pretrained('YituTech/conv-bert-base')

All model checkpoint layers were used when initializing TFConvBertModel.

All the layers of TFConvBertModel were initialized from the model checkpoint at YituTech/conv-bert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFConvBertModel for predictions without further training.


In [17]:
# concating Numerical with LSTM by changing numerical shape, the main now model

from tensorflow.keras.layers import Input, Dense, Dropout, Bidirectional, LSTM, GlobalAveragePooling1D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_end_to_end_convBert_model_main(convBert_model, num_numerical_features, max_len=512):


    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    numerical_input = Input(shape=(num_numerical_features,), name="numerical_input")


    convBert_outputs = convBert_model(input_word_ids, attention_mask=input_mask, token_type_ids=segment_ids)
    convBert_sequence_output = convBert_outputs[0]


    numerical_output = Dense(max_len, activation='relu')(numerical_input)
    numerical_output = Dropout(0.2)(numerical_output)


    numerical_output = tf.expand_dims(numerical_output, 1)


    numerical_output = tf.tile(numerical_output, [1, max_len, 1])


    concatenated = concatenate([convBert_sequence_output, numerical_output], axis=-1)

    lstm_output = Bidirectional(LSTM(256, return_sequences=True))(concatenated)
    lstm_output = Bidirectional(LSTM(128))(lstm_output)


    lay = Dense(128, activation='relu')(lstm_output)
    lay = Dropout(0.2)(lay)
    out = Dense(4, activation='softmax')(lay)
    # 0.00001 learning rate had fluctuation, so change it to 0.000001
    model = Model(inputs=[input_word_ids, input_mask, segment_ids, numerical_input], outputs=out)
    model.compile(Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [18]:
num_numerical_features = 6
max_len = 512
# convBert_model = build_convBert_model(model, num_numerical_features, max_len=max_len)
convBert_model = build_end_to_end_convBert_model_main(convBert_model_layer, num_numerical_features, max_len=max_len)


In [19]:
convBert_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 numerical_input (InputLayer)   [(None, 6)]          0           []                               
                                                                                                  
 dense_3 (Dense)                (None, 512)          3584        ['numerical_input[0][0]']        
                                                                                                  
 dropout_77 (Dropout)           (None, 512)          0           ['dense_3[0][0]']                
                                                                                                  
 input_word_ids (InputLayer)    [(None, 512)]        0           []                               
                                                                                            

In [ ]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1, min_lr=0.000001)

train_sh = convBert_model.fit(
    albert_train_input, y_train,
    validation_data=(albert_validation_input, y_validation),
    epochs=35,
    callbacks=[earlystopping, reduce_lr],
    batch_size=8,
    verbose=1,
)

In [ ]:

results = convBert_model.evaluate(albert_test_input, test_labels, verbose=0)


print('Test accuracy:', results[1])

In [ ]:
from sklearn.metrics import classification_report


predicted_labels = convBert_model.predict(albert_test_input)


predicted_labels = np.argmax(predicted_labels, axis=1)

test_labels_decoded = np.argmax(test_labels, axis=1)

print(classification_report(test_labels_decoded, predicted_labels))